In [1]:
# Combined Memory

In [2]:
import openai
from openai import OpenAI
import langchain
import os
# Step 1: Load environment variables
%reload_ext dotenv
%dotenv
# Step 3: Access your OpenAI API key from environment variables
for key, value in os.environ.items():
    if key == "OPENAI_API_KEY":
        print(f"{key}: value")

OPENAI_API_KEY: value


In [3]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
from langchain_openai.chat_models import ChatOpenAI
chat = ChatOpenAI(model='gpt-4o-mini', temperature=0,max_completion_tokens=50)
# response = chat.invoke("I have recently adopted a dog. Could you suggest some dog names?")
# print(response.content)

In [6]:
from langchain_openai.chat_models import ChatOpenAI

from langchain_core.messages import SystemMessage
from langchain_core.prompts import (ChatPromptTemplate, 
                                    HumanMessagePromptTemplate, 
                                    MessagesPlaceholder,
                                    PromptTemplate)

from langchain.chains.llm import LLMChain

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import (ConversationBufferMemory,
                              ConversationBufferWindowMemory,
                              ConversationSummaryMemory,
                              CombinedMemory)

from langchain.globals import set_verbose
set_verbose(True)

In [7]:
TEMPLATE = '''
The following is a friendly conversation between a human and an AI. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Past messages:
{message_buffer_log}

Conversation summary:
{message_summary_log}

Human: {question}
AI:
'''

In [8]:
prompt_template = PromptTemplate.from_template(template=TEMPLATE)

In [9]:
chat_buffer_memory = ConversationBufferMemory(memory_key='message_buffer_log',
                                              input_key='question',return_messages=False)

chat_summary_memory = ConversationSummaryMemory(llm=chat, memory_key='message_summary_log',
                                                input_key='question', return_messages=False)

chat_combined_memory = CombinedMemory(memories=[chat_buffer_memory,chat_summary_memory])

C:\Users\Abdelrahman\AppData\Local\Temp\ipykernel_21764\1048753577.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  chat_buffer_memory = ConversationBufferMemory(memory_key='message_buffer_log',
C:\Users\Abdelrahman\AppData\Local\Temp\ipykernel_21764\1048753577.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  chat_summary_memory = ConversationSummaryMemory(llm=chat, memory_key='message_summary_log',


In [10]:
chat_combined_memory.load_memory_variables({})

{'message_buffer_log': '', 'message_summary_log': ''}

In [11]:
chain = LLMChain(llm=chat,prompt=prompt_template,memory=chat_combined_memory)

C:\Users\Abdelrahman\AppData\Local\Temp\ipykernel_21764\2929776012.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=chat,prompt=prompt_template,memory=chat_combined_memory)


In [12]:
reponse = chain.invoke({'question' : "Can you give me an interesting fact I probably didn't know about?"})
reponse



> Entering new LLMChain chain...
Prompt after formatting:

The following is a friendly conversation between a human and an AI. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Past messages:


Conversation summary:


Human: Can you give me an interesting fact I probably didn't know about?
AI:



> Entering new LLMChain chain...
Prompt after formatting:
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help human

{'question': "Can you give me an interesting fact I probably didn't know about?",
 'message_buffer_log': '',
 'message_summary_log': '',
 'text': 'Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible!'}

In [13]:
print(reponse['text'])

Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible!


In [14]:
reponse = chain.invoke({'question' : "Can you give me another interesting fact I probably didn't know about?"})
reponse



> Entering new LLMChain chain...
Prompt after formatting:

The following is a friendly conversation between a human and an AI. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Past messages:
Human: Can you give me an interesting fact I probably didn't know about?
AI: Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible!

Conversation summary:
The human asks the AI for an interesting fact. The AI shares that honey never spoils, citing that archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible.

Human: Can you give me another interesting fact I probably didn't know about?
AI:



> Entering new LLMChain chain...
Prompt after formatting:
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summ

{'question': "Can you give me another interesting fact I probably didn't know about?",
 'message_buffer_log': "Human: Can you give me an interesting fact I probably didn't know about?\nAI: Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible!",
 'message_summary_log': 'The human asks the AI for an interesting fact. The AI shares that honey never spoils, citing that archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible.',
 'text': 'Did you know that octopuses have three hearts? Two pump blood to the gills, where it gets oxygen, and the third pumps it to the rest of the body. When they swim, the heart that delivers blood to the body actually stops'}

In [15]:
print(reponse['text'])

Did you know that octopuses have three hearts? Two pump blood to the gills, where it gets oxygen, and the third pumps it to the rest of the body. When they swim, the heart that delivers blood to the body actually stops


In [16]:
reponse = chain.invoke({'question' : "Can you talk about Rodri?"})
reponse



> Entering new LLMChain chain...
Prompt after formatting:

The following is a friendly conversation between a human and an AI. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Past messages:
Human: Can you give me an interesting fact I probably didn't know about?
AI: Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible!
Human: Can you give me another interesting fact I probably didn't know about?
AI: Did you know that octopuses have three hearts? Two pump blood to the gills, where it gets oxygen, and the third pumps it to the rest of the body. When they swim, the heart that delivers blood to the body actually stops

Conversation summary:
The human asks the AI for an interesting fact. The AI shares that honey never spoils, citing that archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still 

{'question': 'Can you talk about Rodri?',
 'message_buffer_log': "Human: Can you give me an interesting fact I probably didn't know about?\nAI: Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible!\nHuman: Can you give me another interesting fact I probably didn't know about?\nAI: Did you know that octopuses have three hearts? Two pump blood to the gills, where it gets oxygen, and the third pumps it to the rest of the body. When they swim, the heart that delivers blood to the body actually stops",
 'message_summary_log': 'The human asks the AI for an interesting fact. The AI shares that honey never spoils, citing that archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible. The human then',
 'text': 'Rodri, whose full name is Rodrigo Hernández Cascante, is a Spanish professional footballer who plays as a defensive

In [17]:
print(reponse['text'])

Rodri, whose full name is Rodrigo Hernández Cascante, is a Spanish professional footballer who plays as a defensive midfielder. He is known for his excellent passing ability, tactical awareness, and physical presence on the field. Rodri began his career at


In [18]:
chat_buffer_memory.load_memory_variables({})

{'message_buffer_log': "Human: Can you give me an interesting fact I probably didn't know about?\nAI: Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible!\nHuman: Can you give me another interesting fact I probably didn't know about?\nAI: Did you know that octopuses have three hearts? Two pump blood to the gills, where it gets oxygen, and the third pumps it to the rest of the body. When they swim, the heart that delivers blood to the body actually stops\nHuman: Can you talk about Rodri?\nAI: Rodri, whose full name is Rodrigo Hernández Cascante, is a Spanish professional footballer who plays as a defensive midfielder. He is known for his excellent passing ability, tactical awareness, and physical presence on the field. Rodri began his career at"}

In [19]:
chat_summary_memory.load_memory_variables({})

{'message_summary_log': 'The human asks the AI for an interesting fact, and the AI shares that honey never spoils, noting that archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible. The human'}

In [20]:
chat_combined_memory.load_memory_variables({})

{'message_buffer_log': "Human: Can you give me an interesting fact I probably didn't know about?\nAI: Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible!\nHuman: Can you give me another interesting fact I probably didn't know about?\nAI: Did you know that octopuses have three hearts? Two pump blood to the gills, where it gets oxygen, and the third pumps it to the rest of the body. When they swim, the heart that delivers blood to the body actually stops\nHuman: Can you talk about Rodri?\nAI: Rodri, whose full name is Rodrigo Hernández Cascante, is a Spanish professional footballer who plays as a defensive midfielder. He is known for his excellent passing ability, tactical awareness, and physical presence on the field. Rodri began his career at",
 'message_summary_log': 'The human asks the AI for an interesting fact, and the AI shares that honey never spoils, noting that archaeol

In [22]:
print(chat_combined_memory.load_memory_variables({})['message_summary_log'])

The human asks the AI for an interesting fact, and the AI shares that honey never spoils, noting that archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible. The human
